In [44]:
import pandas as pd
import numpy as np
import random
import json
from datetime import datetime

In [45]:
df_am = pd.read_csv('data/amazon_mobiles/products.csv')
df_a = pd.read_csv('data/amazon/products.csv')
df_e = pd.read_csv('data/ecommerce/products.csv')
df_f = pd.read_csv('data/flipkart/products.csv')
df_j = pd.read_csv('data/jcpenney/products.csv')

In [46]:
am_prices = {}
a_prices = {}
e_prices = {}
f_prices = {}
j_prices = {}

def getIdPrice(id, price, store_obj):
    store_obj[id] = price

df_am.apply(lambda x: getIdPrice(x['id'], x['price'], am_prices), axis=1)
df_a.apply(lambda x: getIdPrice(x['id'], x['price'], a_prices), axis=1)
df_e.apply(lambda x: getIdPrice(x['id'], x['price'], e_prices), axis=1)
df_f.apply(lambda x: getIdPrice(x['id'], x['price'], f_prices), axis=1)
df_j.apply(lambda x: getIdPrice(x['id'], x['price'], j_prices), axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
10609    None
10610    None
10611    None
10612    None
10613    None
Length: 10614, dtype: object

In [47]:
ids = []
df_am['id'].apply(lambda x: ids.append(x))
df_a['id'].apply(lambda x: ids.append(x))
df_e['id'].apply(lambda x: ids.append(x))
df_f['id'].apply(lambda x: ids.append(x))
df_j['id'].apply(lambda x: ids.append(x))

0        None
1        None
2        None
3        None
4        None
         ... 
10609    None
10610    None
10611    None
10612    None
10613    None
Name: id, Length: 10614, dtype: object

In [48]:
df = pd.DataFrame()
df.insert(0, 'id', range(0, 300000))
df['client'] = np.random.randint(1, 100000, df.shape[0])

In [49]:
def chooseRandomProducts(row):
    n = random.choice(range(1, 10))
    products = random.choices(ids, k=n)
    return products

df['products'] = df.apply(chooseRandomProducts, axis=1)

In [50]:
def calculateTotal(row):
    products = row.products
    total = 0
    for pr in products:
        if pr in am_prices.keys():
            total += am_prices[pr]
        elif pr in a_prices.keys():
            total += a_prices[pr]
        elif pr in e_prices.keys():
            total += e_prices[pr]
        elif pr in f_prices.keys():
            total += f_prices[pr]
        elif pr in j_prices.keys():
            total += j_prices[pr]
        else:
            total += 0
    return "{:.2f}".format(total)

df['total'] = df.apply(calculateTotal, axis=1)

In [51]:
df

,id,client,products,total
0,0,12273,"[fabcd1dc-62ec-421b-ac56-4947dd62d0fb, 957ecbc...",2052.93
1,1,63432,"[3e49b68b164de5765d2a6306c692c7bc, cc90d1ed846...",6502.34
2,2,43544,"[9527fdf4-791c-422d-9ca1-86adb4f9e8c2, 22ec412...",138.24
3,3,56051,"[c79822d1a61d3002496383d0098285ea, f29565d5-be...",2677.64
4,4,15449,"[5d4c2f94fce965b9b47d4529eea54211, 73da7ac34ce...",314.65
...,...,...,...,...
299995,299995,88176,"[3e075c69-7107-4169-9dd4-3975900881bb, 9dc1919...",14033.21
299996,299996,29260,"[94e76486070bd23025c6d7b829e45bd1, f0a17bb5-e6...",193.00
299997,299997,57567,"[3996fc5d-9559-494a-9aa8-b82eca2a5566, d41d354...",1934.39
299998,299998,81888,"[b8e4ed37f99d6d064c225297ad63343c, bb8b0979-73...",2713.80


In [52]:
def parseDate(date):
    date = date.replace('Z', '')
    test_date = datetime.strptime(date, "%Y-%m-%d %X")
    return test_date.isoformat()

In [53]:
date_df = pd.read_csv('data/dates.csv')
date_df['date'] = date_df['date'].map(lambda x: parseDate(x))
dates = date_df.date

In [54]:
df['date'] = np.random.choice(dates, size=df.shape[0])

In [55]:
df

,id,client,products,total,date
0,0,12273,"[fabcd1dc-62ec-421b-ac56-4947dd62d0fb, 957ecbc...",2052.93,2021-08-11T17:00:50
1,1,63432,"[3e49b68b164de5765d2a6306c692c7bc, cc90d1ed846...",6502.34,2021-06-08T18:13:25
2,2,43544,"[9527fdf4-791c-422d-9ca1-86adb4f9e8c2, 22ec412...",138.24,2021-04-11T22:18:18
3,3,56051,"[c79822d1a61d3002496383d0098285ea, f29565d5-be...",2677.64,2021-04-21T15:51:05
4,4,15449,"[5d4c2f94fce965b9b47d4529eea54211, 73da7ac34ce...",314.65,2022-04-21T00:52:07
...,...,...,...,...,...
299995,299995,88176,"[3e075c69-7107-4169-9dd4-3975900881bb, 9dc1919...",14033.21,2020-09-19T14:23:17
299996,299996,29260,"[94e76486070bd23025c6d7b829e45bd1, f0a17bb5-e6...",193.00,2019-12-02T11:58:26
299997,299997,57567,"[3996fc5d-9559-494a-9aa8-b82eca2a5566, d41d354...",1934.39,2021-02-26T12:02:09
299998,299998,81888,"[b8e4ed37f99d6d064c225297ad63343c, bb8b0979-73...",2713.80,2022-01-29T08:13:26


In [ ]:
result = df.to_json(orient="records", default_handler=str)
parsed = json.loads(result)
json_string = json.dumps(parsed, indent=4)
with open('data/purchases.json', 'w') as outfile:
    outfile.write(json_string)